# Investment Planning in PyPSA

This tutorial covers how to perform investment planning in PyPSA, including multi-period optimization, investment costs, and capacity expansion planning.

## Multi-Period Investment Planning

Let's create a network with multiple investment periods to plan capacity expansion over time:

In [ ]:
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create network
network = pypsa.Network()

# Define investment periods
years = [2020, 2030, 2040, 2050]
freq = "24"  # 24-hour resolution

# Create snapshots for each period
snapshots = pd.DatetimeIndex([])
for year in years:
    # Calculate integer number of periods
    num_periods = int(8760 / int(freq))
    period = pd.date_range(
        start=f"{year}-01-01 00:00",
        freq=f"{freq}h",
        periods=num_periods
    )
    snapshots = snapshots.append(period)

# Set snapshots and investment periods
network.snapshots = pd.MultiIndex.from_arrays([snapshots.year, snapshots])
network.investment_periods = years

print("Network snapshots:")
print(network.snapshots)
print("\nInvestment periods:")
print(network.investment_periods)

## Setting Up Investment Weightings

We need to set up investment period weightings to account for the time value of money:

In [ ]:
# Set investment period weightings
r = 0.01  # Social discount rate
T = 0

# First, set the years for each investment period
# Assuming 10-year periods between each investment year
network.investment_period_weightings["years"] = {
    2020: 10,  # 2020-2030
    2030: 10,  # 2030-2040
    2040: 10,  # 2040-2050
    2050: 10   # 2050-2060
}

# Calculate objective function weightings based on discount rate
for period, nyears in network.investment_period_weightings.years.items():
    discounts = [(1 / (1 + r) ** t) for t in range(T, T + int(nyears))]
    network.investment_period_weightings.at[period, "objective"] = sum(discounts)
    T += int(nyears)

print("Investment period weightings:")
print(network.investment_period_weightings)

## Adding Network Components

Let's add components with investment options:

In [3]:
# Add buses
for i in range(3):
    network.add("Bus", f"bus {i}")

# Add carriers
network.add("Carrier", "gas", co2_emissions=0.2)
network.add("Carrier", "solar")
network.add("Carrier", "wind")

# Add generators with investment options
network.add(
    "Generator",
    "gas_plant",
    bus="bus 0",
    p_nom=0,  # Start with zero capacity
    p_nom_extendable=True,  # Allow capacity expansion
    p_nom_min=0,  # Minimum capacity
    p_nom_max=1000,  # Maximum capacity
    marginal_cost=50,
    capital_cost=1000,
    carrier="gas",
    efficiency=0.4
)

# Create a solar profile for each snapshot in the network
base_solar_profile = [0.0, 0.0, 0.0, 0.0, 0.1, 0.3, 0.5, 0.7, 0.8, 0.9, 0.9, 0.8,
                      0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0]

# Repeat the daily profile for each day in the snapshots
solar_profile = []
for _ in range(len(network.snapshots) // 24):
    solar_profile.extend(base_solar_profile)

# If there are remaining hours, add them
remaining_hours = len(network.snapshots) % 24
if remaining_hours > 0:
    solar_profile.extend(base_solar_profile[:remaining_hours])

# Create the Series with the correct index
solar_profile = pd.Series(solar_profile, index=network.snapshots)

network.add(
    "Generator",
    "solar",
    bus="bus 1",
    p_nom=0,
    p_nom_extendable=True,
    p_nom_min=0,
    p_nom_max=1000,
    p_max_pu=solar_profile,
    marginal_cost=0,
    capital_cost=800,
    carrier="solar"
)

# Add load with growth
base_load_profile = [0.6, 0.5, 0.4, 0.4, 0.5, 0.7, 0.8, 0.9, 0.9, 0.8, 0.7, 0.7,
                     0.7, 0.7, 0.7, 0.7, 0.8, 0.9, 0.9, 0.8, 0.7, 0.6, 0.5, 0.5]

# Repeat the daily profile for each day in the snapshots
load_profile = []
for _ in range(len(network.snapshots) // 24):
    load_profile.extend(base_load_profile)

# If there are remaining hours, add them
remaining_hours = len(network.snapshots) % 24
if remaining_hours > 0:
    load_profile.extend(base_load_profile[:remaining_hours])

# Create the Series with the correct index
load_profile = pd.Series(load_profile, index=network.snapshots)

network.add(
    "Load",
    "load",
    bus="bus 2",
    p_set=50 * load_profile * (1 + 0.02 * (network.snapshots.get_level_values(0) - 2020))
)

# Add transmission lines with investment options
for i in range(3):
    network.add(
        "Line",
        f"line_{i}",
        bus0=f"bus {i}",
        bus1=f"bus {(i + 1) % 3}",
        s_nom=0,  # Start with zero capacity
        s_nom_extendable=True,  # Allow capacity expansion
        s_nom_min=0,  # Minimum capacity
        s_nom_max=1000,  # Maximum capacity
        capital_cost=1000,
        x=0.1,
        r=0.01
    )

## Adding Global Constraints

Let's add some global constraints for the investment planning:

In [4]:
# Add CO2 emission limit that becomes stricter over time
for year in years:
    network.add(
        "GlobalConstraint",
        f"co2_limit_{year}",
        sense="<=",
        constant=1000 * (1 - 0.1 * (year - 2020)),  # 10% reduction per decade
        investment_period=year
    )

# Add renewable share target
for year in years:
    network.add(
        "GlobalConstraint",
        f"renewable_share_{year}",
        sense=">=",
        constant=0.3 + 0.1 * (year - 2020),  # 10% increase per decade
        investment_period=year
    )

## Solving the Investment Planning Problem

Now let's solve the multi-period investment planning problem:

In [ ]:
# Solve the investment planning problem
network.lopf()

# Print optimal capacities for each investment period
print("\nOptimal Generator Capacities:")
print(network.generators.p_nom_opt)

print("\nOptimal Line Capacities:")
print(network.lines.s_nom_opt)

## Analyzing Investment Results

Let's analyze the investment planning results:

In [ ]:
# Plot capacity expansion over time
plt.figure(figsize=(10, 6))
network.generators.p_nom_opt.plot(kind='bar')
plt.title('Optimal Generator Capacities by Investment Period')
plt.xlabel('Generator')
plt.ylabel('Capacity (MW)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Plot line capacity expansion
plt.figure(figsize=(10, 6))
network.lines.s_nom_opt.plot(kind='bar')
plt.title('Optimal Line Capacities by Investment Period')
plt.xlabel('Line')
plt.ylabel('Capacity (MVA)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Print system costs
print("\nTotal System Costs:")
print(network.objective)

## Key Investment Planning Concepts

1. **Investment Periods**
   - Define time periods for investment decisions
   - Can have different constraints for each period
   - Account for time value of money through discounting

2. **Capacity Expansion**
   - Components can be made extendable
   - Set minimum and maximum capacity limits
   - Define capital costs for investment

3. **Global Constraints**
   - Can be period-specific
   - Examples: CO2 limits, renewable share targets
   - Can guide investment decisions

4. **Cost Components**
   - Capital costs for investment
   - Operational costs (marginal costs)
   - Discounted over time

## Next Steps

In the next tutorial, we'll explore storage systems and demand-side management options.